## Causal Inference
- 인과추론의 근본적인 문제
    - 한 개인의 counterfactual은 관측 불가
- selection bias 존재 (선택편향)
    - 처치를 받은 집단과 안받은 집단을 단순비교하면, 원래부터 다른 특성 차이까지 같이 섞여 진짜 효과를 알 수 없음
    - 선택편향 = $E[Y(0)∣T=1]−E[Y(0)∣T=0]$, 처치군이 처치를 받지 않았더라도 평균적으로 통제군과 같았을까?
        - $E[Y(0)∣T=1]>E[Y(0)∣T=0]$일 수 있음 즉, 처치군은 애초에 outcome이 높을 집단 (아래 데이터셋에 의거)
        - **RCT**: $E[Y(0)∣T=1]=E[Y(0)∣T=0]$ 즉, 선택편향 = 0 &rarr; 단순 평균 차이 = ATE
        - **Observational Data**: 교란변수로 인해 $E[Y(0)∣T=1] != E[Y(0)∣T=0]$    &rarr; 선택편향 발생
            - 단순평균차이 = 진짜효과 + 원래 집단 차이(선택편향)
            - 인과추론 방법은 **"원래 집단 차이(선택편향)"를 제거하는 것**이 목표
- **Observation Data에서 $ATE=E[Y(1)−Y(0)]$을 알아보자**

**인과추론 방법 정리**
| 방법 | 언제 사용? | 기본 추정 대상 | 기본 추정 아이디어 | CATE 가능? | 내가 생각하는 핵심 |
|------|------------|---------------|-------------------|------------|--------------------|
| **Regression Adjustment (RA)** | 관측 데이터, unconfoundedness 가정 가능할 때 | ATE | Y ~ T + X 회귀 | interaction 넣으면 가능 | 모델 misspecification에 매우 취약 |
| **Propensity Score / IPW** | treatment selection bias가 X로 설명 가능할 때 | ATE | 1/e(X)로 가중 평균 | stratification/weight별 가능 | extreme weight가 가장 큰 위험 |
| **DML** | 고차원 X, ML 사용 필요할 때 | ATE 또는 CATE | nuisance model 제거 후 orthogonalization | ✅ 기본 출력 | 현대 causal ML의 표준 |
| **IV** | 숨은 교란 존재, instrument 있을 때 | LATE | 2SLS (Z → T → Y) | interaction 넣으면 가능 | 항상 “complier”에 대한 효과 |
| **DiD** | 패널 데이터, 정책 전후 비교 가능 | ATE | (Treat×Post) 계수 | triple interaction 필요 | parallel trend 가정이 생명 |
| **RDD** | cutoff 기반 정책 | cutoff 근처 LATE | local polynomial | interaction 필요 | local effect라는 점이 핵심 |

### 1. Potential Outcome Framework
- **목적: 인과효과를 정확히 정의하는 법 배우기**
    - Y(1), Y(0), ATE, ATT, 관측가능성과 반사실

In [24]:
import numpy as np
import pandas as pd

In [25]:
np.random.seed(42)
n = 5000

# 공변량 (X)
age = np.random.normal(40,10,n)
income = np.random.normal(5000, 1000, n)

# treatment 확률 (confounding존재)
prob_test = 1 / (1 + np.exp(-(0.05*age - 0.0005*income))) # 교란변수 존재를 위해, 교란변수 없으려면 prob_test= 0.5
treatment = np.random.binomial(1, prob_test)

# 진짜 인과효과 (가정)
true_effect = 5

outcome = (
    0.3*age
    + 0.01*income
    + true_effect*treatment
    + np.random.normal(0,5,n)
)

df = pd.DataFrame({
    "age": age,
    "income": income,
    "treatment": treatment,
    "outcome": outcome
})

In [26]:
df.head()

,age,income,treatment,outcome
0,44.967142,4576.240318,0,72.932346
1,38.617357,4546.585892,0,59.894914
2,46.476885,3204.356827,1,47.837203
3,55.230299,4669.909808,1,70.007768
4,37.658466,5732.829082,0,77.529024


### 2. Confounding 확인
- treatment, control 집단의 평균 나이 및 수입이 다름 -> 교란 존재

In [27]:
df.groupby('treatment')[['age','income']].mean()

,age,income
treatment,,
0,38.253704,5181.255804
1,42.766261,4702.713739


## **관측된 교란변수만 존재하는 경우**

### 3. Regression Adjustment - Backdoor 차단방법 1
- **목적: Backdoor을 차단하는 방법, 같은 X끼리 비교하자 (confounder 통제)**
    - Backdoor = 우리가 알고싶은건 $T$ &rarr; $Y$, 하지만 $T$ &larr; $X$ &rarr; $Y$가 있을 수 있음
        이때 $X$는 교란변수이고, T에서 시작해서 X로 가고 Y로 가는 경로를 backdoor라고 함
- Ignorability 가정: (Y(1),Y(0))⊥T∣X
    - X가 주어지면, treatment배정은 잠재결과와 독립이다. 즉, 같은 X를 가진 사람들끼리는 treatment가 무작위처럼 배정됨
    - **숨겨진 교란**이 없다는 가정하에 가능
- 해당 가정이 성립해야하는 이유
    - 해당 데이터는 관측 데이터이기 때문에 개인의 Y(1), Y(0)을 동시에 볼 수 없음
- 핵심 : confounder(교란변수)를 통제하여 treatment의 순수 효과만 분리하는 방법, 즉 x를 고정하고 t만 바꾸자
    - e.g. age와 income을 고정했을 때 treatment가 outcome을 얼마나 변화시키는가?
- 수학적 구조
    - outcome 생성식 : Y=0.3*age+0.01*income+5*T+ϵ
    - 우리가 추정하고자 하는 모형 : Y=α*T+β1*age+β2*income+ϵ / 이때 α가 ATE 추정치
    - OLS : 해당 식에서 실제 Y값(outcome)과 예측 Y값의 오차가 가장 최소가 되도록 하는 α, β1, β2값을 찾자
- ATE
    - $ATE=E_X[m1​(X)−m0​(X)]$
        - $m_t​(X)$ = 회귀는 $m_t​$을 추정하는 모델

In [28]:
import statsmodels.api as sm

X = df[['treatment','age','income']]
X = sm.add_constant(X)

model = sm.OLS(df['outcome'], X).fit()
print(model.summary())

# treatment의 계수가 5.2로 나옴 -> 데이터셋의 인과효과를 5로 설정했으므로 타당

                            OLS Regression Results                            
Dep. Variable:                outcome   R-squared:                       0.818
Model:                            OLS   Adj. R-squared:                  0.818
Method:                 Least Squares   F-statistic:                     7470.
Date:                Tue, 24 Feb 2026   Prob (F-statistic):               0.00
Time:                        15:21:36   Log-Likelihood:                -15158.
No. Observations:                5000   AIC:                         3.032e+04
Df Residuals:                    4996   BIC:                         3.035e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.1539      0.465     -2.482      0.0

### 3. Propensity Score/IPW - Backdoor 차단방법 2
- **목적: treatment 확률을 구하고(Propensity), 그 확률을 이용해 인과효과 추정(IPW)**
- Propensity Score(PS)
    - 특성 X에서 treatment 받을 확률 $e(X)=P(T=1∣X)$
    - confounding을 요약하는 변수 역할을 함 &larr; x 전체를 통제하는 대신 e(x)하나만 통제해도 충분하다
- IPW(Inverse Probability Weighting)
    - PS의 역수를 가중치로 써서 데이터를 재구성하자 (가짜 RCT처럼 만들기)
        - 가중치: $weight=1/e(x)$, treatment를 무작위처럼 보이게하기 위함
- ATE
    - $ATE=E[TY​/e(X)]−E[(1−T)Y​/1−e(X)]$

In [29]:
from sklearn.linear_model import LogisticRegression

ps_model = LogisticRegression()
ps_model.fit(df[['age','income']], df['treatment'])

df['ps'] = ps_model.predict_proba(df[['age','income']])[:,1]

In [30]:
# IPW 계산
ate_ipw = (
    (df["treatment"]*df["outcome"]/df["ps"]) # T=1인 사람만 남음 + 그사람의 Y(결괴)를 1/e(x)로 확대
    - ((1-df["treatment"])*df["outcome"]/(1-df["ps"])) # T=0만 남음 + 1/(1-e(x))
).mean()

print(ate_ipw)
# 데이터셋의 인과효과 5로 설정 -> 타당

5.427510896750453


### 4. Double Machine Learning
- **목적: ML을 이용한 인과효과 추정(처치와 결과 각각에서 공변량의 영향을 제거한 후, 순수하게 처치만 바꿨을때 결과가 얼마나 달라지는지 확인)**
    - 필요성: 현실에서는 선형아님, 변수많음, 함수형태모름의 가능성이 존재하기 때문,
            머신러닝으로 예측하면 잘 맞지만 인과효과 왜곡 가능성 존재
- 방법
    - 원하는 건 $Y=θT+g(X)+ε$
        - $θ$ = 알고싶은 인과효과 (ATE)
        - $g(X)$ = 공변량 영향
        - $ε$ = 오차
        - 문제는, $g(X)$가 복잡함
    - 따라서,
      1. Step1 : T에서 X의 영향 제거
        - 머신러닝으로 예측 후, 잔차 계산
            - $\hat{T}=m(X)$ -> $\tilde{T}=T−\hat{T}$
      2. Step2 : Y에서 X의 영향 제거
         - 마찬가지로
            - $\hat{Y}=g(X)$ -> $\tilde{Y}=Y−\hat{Y}$
      3. Step3 : 잔차끼리 회귀
         - $\hat{Y}=θ\tilde{T}$
         - 이때, θ가 바로 인과효과(ATE)

In [31]:
from econml.dml import LinearDML
from sklearn.ensemble import RandomForestRegressor

est = LinearDML(
    model_y = RandomForestRegressor(),
    model_t = RandomForestRegressor()
)

est.fit(
    Y=df['outcome'],
    T=df['treatment'],
    X=df[['age','income']]
)

print('ATE:', est.ate(df[['age','income']]))

ATE: 5.0181352686786


In [32]:
# CATE 추정 (각 관측치별 추정 인과효과)
cate = est.effect(df[['age','income']])
df['cate'] = cate

In [33]:
df.head()

,age,income,treatment,outcome,ps,cate
0,44.967142,4576.240318,0,72.932346,0.502031,5.080452
1,38.617357,4546.585892,0,59.894914,0.426191,5.118149
2,46.476885,3204.356827,1,47.837203,0.690007,5.360245
3,55.230299,4669.909808,1,70.007768,0.617298,5.009944
4,37.658466,5732.829082,0,77.529024,0.275930,4.874498


## **미관측 교란변수가 존재하는 경우**

### 5. Instrumental Variable(2SLS)
- **목적: 히든 교란변수가 있을 때의 최후 무기**
    - 필요성: 히든교란변수는 treatment와 outcome 둘다 영향을 미침 &rarr; 편향 발생
        따라서, 교란을 제거한다기보다 교란과 독립인 variation만 사용한다
- 방법
    - Instrumental Variable
        - treatment에는 영향을 주지만, outcome에는 직접 영향이 없는 변수를 찾는다 = **Instrument(도구변수)**
        - ```
          # 현재구조
          hidden -> treatment -> outcome
          hidden -> outcome
          instrument -> treatment
          ```
        - instrument는 hidden 과 독립
        - 따라서, instrument를 통해 변한 treatment 부분은 히든의 영향이 제거된 꺠끗한 변수임 &rarr; 인과효과 추정
    - 2SLS
        - step1) instrument로 treatment 예측 (hidden confounder 영향이 제거된 부분)
            - Z로 예측된 T는 hidden영향이 없음
            - $T=πZ+error$ &rarr; $\hat{T}$
        - step2) 예측된 treatment만 사용해서 회귀
            -  $Y=β\hat{T}+error$
            - 이때, β가 IV추정치
        - hidden 문제를 해결하는 이유?
            - instrument는 hidden과 독립
                - $Cov(Z, hidden) = 0$  (공분산)
            - $Z$를 통해 생성된 T의 variation은 hidden의 영향이 없음
    - 해석
        - IV의 해석은 ATE가 아닌, LATE임
            - instrument에 의해 treatment가 바뀌는 집단에 대한 효과


In [34]:
# data load
instrument = np.random.binomial(1,0.5,n)
hidden = np.random.normal(0,1,n)
treatment = (0.5*instrument + 0.8*hidden > 0).astype(int)
outcome = 4*treatment + 3*hidden + np.random.normal(0,1,n)

In [35]:
df_iv = pd.DataFrame(
    {'outcome': outcome,
     'treatment': treatment,
     'instrument': instrument
     })

In [36]:
df_iv.head()

,outcome,treatment,instrument
0,3.460213,1,1
1,5.912310,1,1
2,7.942380,1,1
3,9.263438,1,0
4,7.212351,1,1


In [40]:
from linearmodels.iv import IV2SLS

iv_model = IV2SLS.from_formula(
    'outcome ~ 1 + [treatment ~ instrument]',
    df_iv
).fit()
# 종속변수 ~ 외생변수(일반 독립변수) + [내생변수(교란으로 문제되는 변수)+도구변수]

print(iv_model.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                outcome   R-squared:                      0.5648
Estimator:                    IV-2SLS   Adj. R-squared:                 0.5647
No. Observations:                5000   F-statistic:                    128.60
Date:                Tue, Feb 24 2026   P-value (F-stat)                0.0000
Time:                        15:23:20   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -0.1057     0.2252    -0.4694     0.6388     -0.5471      0.3357
treatment      4.1260     0.3638     11.340     0.00

### 6. Difference-in-Differences
- **목적: 어떤 정책/처치가 특정 시점 이후에 특정 집단에만 적용되었을 때, 그 효과를 추정함**
    - e.g. 특정 지역에만 프로모션 실시, 특정 시점 이후에 가격 인상
- 방법
공통 변화가 존재함..(계절 효과, 시간 트렌드 등..) 따라서 공통변화는 통제집단 변화로 빼버림
  | 구분      | before | after |
  |---------|--------|-------|
  | control | A      | B     |
  | treat   | C      | D     |
  - 일때, $(D-C) - (B-A)$ 즉 치료집단의 변화 - 통제집단의 변화
  - $(Y_{post,treat}−Y_{pre,treat})−(Y_{post,control}−Y_{pre,control})$ = 정책효과

In [41]:
# data load
df_did = pd.DataFrame({
    "group": np.repeat([0,1],200), # 0=control, 1=treatment
    "post": np.tile(np.repeat([0,1],100),2) # 0=before, 1=after
})

df_did["outcome"] = (
    5*df_did["group"]*df_did["post"]
    + np.random.normal(0,1,len(df_did))
) # group=1 & post=1 인 경우에만 효과 5발생

In [44]:
import statsmodels.formula.api as smf

model = smf.ols(
    "outcome ~ group*post",
    data = df_did
).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                outcome   R-squared:                       0.826
Model:                            OLS   Adj. R-squared:                  0.824
Method:                 Least Squares   F-statistic:                     625.6
Date:                Wed, 25 Feb 2026   Prob (F-statistic):          7.93e-150
Time:                        09:41:11   Log-Likelihood:                -574.92
No. Observations:                 400   AIC:                             1158.
Df Residuals:                     396   BIC:                             1174.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0782      0.102      0.764      0.4

In [49]:
df

,age,income,treatment,outcome,ps,cate
0,44.967142,4576.240318,0,72.932346,0.502031,5.080452
1,38.617357,4546.585892,0,59.894914,0.426191,5.118149
2,46.476885,3204.356827,1,47.837203,0.690007,5.360245
3,55.230299,4669.909808,1,70.007768,0.617298,5.009944
4,37.658466,5732.829082,0,77.529024,0.275930,4.874498
...,...,...,...,...,...,...
4995,39.510350,6301.102063,1,75.475239,0.237291,4.746316
4996,47.114106,3001.655033,1,49.173429,0.718718,5.399532
4997,71.129102,4294.683276,1,72.772501,0.814209,5.009680
4998,48.080362,5495.765573,0,65.753426,0.422089,4.872461


### 7. Regression Discontinuity
- **목적: 임계값 기반 정책에서 인과효과 추정** (임계값 근처에서 outcome의 점프 크기 = 인과효과)
    - e.g. 시험점수 70점 이상이면 장학금 지급
- 방법 (아래 데이터 예시로..)
    - 핵심 아이디어
      1. 임계값 근처에서는 treatment 받은 사람과 안받은 사람이 거의 동일함 (hidden confouder이 있어도 괜찮은 이유)
      2. 그 근처는 능력차이가 거의 없지만(70점vs69점) treatment는 달라짐 &rarr; 임계값 주변만 보면 거의 랜덤실험과 같음
      3. 따라서, 비슷한 score을 가진 사람들끼리 비교 (score 통제)
      4. 회귀식 = $Y=β_0+β_1T+β_2score+ε$
      4. 인과효과 = $β_1$ : 임계값 근처에서 점프 크기


In [45]:
# data load
score = np.random.uniform(50,90,n) # 50~90 사이 점수 생성, 교란변수
treatment = (score >= 70).astype(int) # 70점 이상이면 treatment = 1, 아니면 0
outcome = 10*treatment + 0.5*score + np.random.normal(0,2,n) # np.random...은 noise
# 중요한건 진짜 인과효과 = 10, 점수자체도 outcome에 영향

band = (score>65)& (score<75)

In [48]:
outcome

array([34.07227048, 51.04429871, 36.15116943, ..., 51.38354543,
       48.58450057, 50.58775882])

In [47]:
outcome[band]

array([34.07227048, 36.15116943, 44.19451774, ..., 45.98832839,
       50.3673115 , 48.58450057])

In [ ]:
X = sm.add_constant(pd.DataFrame({
    'treatment': treatment[band],
    'score': score[band]
}))

sm.OLS(outcome[band])

### 8. 정책 최적화
- **목적: 추정한 인과효과(CATE)를 실제 의사결정에 활용**
    - CATE = 각 개인 i에게 캠페인을 하면 얼마나 이득이 생기는지, 정책적으로는 **앞으로 누구에게 줄지** 결정하기 위한 값
- 방법
    1. 모두에게 treatment를 못주기 때문에 특정 개인들에게만 treatment 줌 &rarr; 정책 최적화
    2. 전체 모집단에서 CATE 높은 사람부터 타겟팅(treatment여부와 상관없음) &rarr; 전체 평균 효과 극대화
       - CATE가 크다 = treatment를 줬을 때 이득이 크다
            - 과거에 treatment를 받지 않아도 cate가 높으면 대상이 됨

In [55]:
df['target'] = df['cate'] > np.percentile(df['cate'], 80) # 상위 20%만 선택

# 실제 타겟팅한 집단의 평균 treatment 효과
policy_gain = df[df['target'] == 1]['cate'].mean()
print(policy_gain)

5.321559991618473


In [54]:
policy_gain

np.float64(5.321559991618473)